In [1]:
import pandas as pd
import pandas as pd
from tqdm import tqdm_notebook as tqdm

#Scikit-learn


#NLTK
import nltk
from nltk.corpus import stopwords

import plotly.express as px

nltk.download("stopwords")
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

from nltk.stem.snowball import SnowballStemmer 
import string
import re
from nltk.stem import *
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation




from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
data = pd.read_csv('./df_sep_label_.csv')

In [3]:
data.head(5)

,sentiment,Классификация по ОКПД2_x,Наименование закупки_x,Реестровый номер закупки,Закупки по,Дата размещения
0,Не подходит,62.01.29.000: Оригиналы программного обеспечен...,Предоставление неисключительного права испол...,№0169300040423000063,44-ФЗ,04.09.2023
1,Подходит,62.09.20.190: Услуги по технической поддержке ...,оказание услуг по обслуживанию и сопровождени...,№0304300010623000024,44-ФЗ,21.09.2023
2,Не подходит,62.02.20.190: Услуги консультативные в области...,Предоставление услуг по сопровождению програм...,№0162300011123000065,44-ФЗ,07.09.2023
3,Не подходит,62.01.29.000: Оригиналы программного обеспечен...,Программное средство антивирусной защиты,№0122100010223000021,44-ФЗ,18.09.2023
4,Не подходит,62.01.29.000: Оригиналы программного обеспечен...,Программное средство антивирусной защиты,№0302300050523000034,44-ФЗ,21.08.2023


In [4]:
data.shape

(3204, 6)

In [5]:
data.rename(columns={'Классификация по ОКПД2_x':'okpd_2'},inplace=True)
data.rename(columns={'Наименование закупки_x':'name'},inplace=True)
data.rename(columns={'Реестровый номер закупки':'number'},inplace=True)
data.rename(columns={'Закупки по':'fz'},inplace=True)
data.rename(columns={'Дата размещения':'date'},inplace=True)


In [6]:
data

,sentiment,okpd_2,name,number,fz,date
0,Не подходит,62.01.29.000: Оригиналы программного обеспечен...,Предоставление неисключительного права испол...,№0169300040423000063,44-ФЗ,04.09.2023
1,Подходит,62.09.20.190: Услуги по технической поддержке ...,оказание услуг по обслуживанию и сопровождени...,№0304300010623000024,44-ФЗ,21.09.2023
2,Не подходит,62.02.20.190: Услуги консультативные в области...,Предоставление услуг по сопровождению програм...,№0162300011123000065,44-ФЗ,07.09.2023
3,Не подходит,62.01.29.000: Оригиналы программного обеспечен...,Программное средство антивирусной защиты,№0122100010223000021,44-ФЗ,18.09.2023
4,Не подходит,62.01.29.000: Оригиналы программного обеспечен...,Программное средство антивирусной защиты,№0302300050523000034,44-ФЗ,21.08.2023
...,...,...,...,...,...,...
3199,Не подходит,62.02.30.000: Услуги по технической поддержке ...,"Оказание услуг по восстановлению, обновлению п...",№32312635470,223-ФЗ,01.08.2023
3200,Не подходит,62.02.30.000: Услуги по технической поддержке ...,Оказание услуг по обслуживанию программы 1С:Бу...,№32312635242,223-ФЗ,01.08.2023
3201,Не подходит,62.03.12.130: Услуги по сопровождению компьюте...,Услуги по адаптации и сопровождению экземпляро...,№32312634646,223-ФЗ,01.08.2023
3202,Не подходит,62.09.2: Услуги в области информационных техно...,Предоставление права использования программы д...,№32312634315,223-ФЗ,01.08.2023


In [7]:
data.isnull().sum()

sentiment    18
okpd_2        2
name          0
number        0
fz            0
date          0
dtype: int64

Мы видим присутствие пропущенных значений, необходимо их удалить.

In [8]:
data.dropna(subset=['sentiment'], inplace=True)
data.dropna(subset=['okpd_2'], inplace=True)

In [9]:
data.isnull().sum()

sentiment    0
okpd_2       0
name         0
number       0
fz           0
date         0
dtype: int64

In [10]:
#Дополнение списка стоп-слов
# Дополнить список

stop_words = stopwords.words('russian')
stop_words.append('енпф')
stop_words.append('«')
stop_words.append('»')

Функция предобработки данных.

In [11]:
def preprocessing(texts):
    new_text = []
    for i in tqdm(range(len(texts))):  # Итерируемся по индексам текстов в списке texts
        text = texts[i]  # Получаем текущий текст по индексу i
        stop_free = " ".join([i for i in text.lower().split() if i not in stop_words]) 
        # Преобразуем текст в нижний регистр, разделяем на слова и удаляем стоп-слова (слова, которые не несут смысловой нагрузки)
        punc_free = ''.join(ch for ch in stop_free if ch not in spec_chars) 
        # Удаляем пунктуацию (символы, не относящиеся к словам)
        lem_text = " ".join([lemma.parse(word)[0].normal_form for word in punc_free.split(' ')]) 
        # Применяем лемматизацию к каждому слову и объединяем обработанные слова обратно в текст
        new_text.append(lem_text) 
        # Добавляем обработанный текст в список new_text
    return new_text  # Возвращаем список new_text, содержащий все обработанные тексты

In [12]:
print(data['sentiment'].unique(), len(data['sentiment'].unique()))

['Не подходит' 'Подходит' 'Скорее нет' 'Скорее да' 'Затрудняюсь'] 5


## Выборка данных

In [13]:
topics = ['Не подходит', 'Подходит', 'Скорее нет', 'Скорее да', 'Затрудняюсь']

In [14]:
df_res = pd.DataFrame()
in_cat_count = data.shape[0]

for topic in tqdm(topics):
    df_topic = data[data['sentiment'] == topic][:in_cat_count]
    df_res = df_res.append(df_topic, ignore_index=True)

C:\Users\Mi\AppData\Local\Temp\ipykernel_24008\1222733655.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for topic in tqdm(topics):


  0%|          | 0/5 [00:00<?, ?it/s]

In [15]:
df_res.shape

(3184, 6)

## Предобработка

In [ ]:
text = prep_preproccessed_name[3180]
text = text.replace('«', '"').replace('»', '"')
print(text)

In [22]:
def remove_punctuation(text):
    text = text.replace('«', '').replace('»', '') # добавил обработку ковычек
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])

def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])


def remove_multiple_spaces(text):
	return re.sub(r'\s+', ' ', text, flags=re.I)

mystem = Mystem() 

russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...'])

def lemmatize_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords and token != " "]
    text = " ".join(tokens)
    return text

In [17]:
df_res['fz'] = df_res['fz'].replace('44-ФЗ', 44)
df_res['fz'] = df_res['fz'].replace('223-ФЗ', 223)

In [23]:
preproccessing = lambda text: (remove_multiple_spaces(remove_numbers(remove_punctuation(text))))
df_res['preproccessed_name'] = list(map(preproccessing, df_res['name']))
df_res['preproccessed_okpd_2'] = list(map(preproccessing, df_res['okpd_2']))

In [24]:
prep_preproccessed_name = [remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))) for text in tqdm(df_res['preproccessed_name'])]
prep_preproccessed_okpd_2 = [remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))) for text in tqdm(df_res['preproccessed_okpd_2'])]

C:\Users\Mi\AppData\Local\Temp\ipykernel_24008\2915617272.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  prep_preproccessed_name = [remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))) for text in tqdm(df_res['preproccessed_name'])]


  0%|          | 0/3184 [00:00<?, ?it/s]

C:\Users\Mi\AppData\Local\Temp\ipykernel_24008\2915617272.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  prep_preproccessed_okpd_2 = [remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))) for text in tqdm(df_res['preproccessed_okpd_2'])]


  0%|          | 0/3184 [00:00<?, ?it/s]

In [27]:
len(prep_preproccessed_name)
prep_preproccessed_name[3180]

'оказание услуг по внедрению мобильного приложения для управления личным кабинетом абонента'

In [29]:
type[prep_preproccessed_name

type[[' предоставление неисключительного права использования антивирусного программного обеспечения kaspersky endpoint security для бизнеса стандартный russian edition', ' предоставление услуг по сопровождению программного комплекса на платформе с', ' программное средство антивирусной защиты', ' программное средство антивирусной защиты', ' программное средство антивирусной защиты', ' программное средство антивирусной защиты', ' программное средство антивирусной защиты', ' программное средство антивирусной защиты', ' услуги по предоставлению в аренду серверных мощностей для размещения на них программных продуктов семейства с бухгалтерия и информационно технологическому сопровождению с ', ' модернизация процессов управления автоматизированной системой диспетчерской службы скорой медицинской помощи необходимые для создания структурированного электронного документа', ' ежемесячные услуги по обслуживанию поддержке и информационно консультативному сопровождению программных продуктов с предпр

In [26]:
len(prep_preproccessed_okpd_2)
prep_preproccessed_okpd_2[3180]

' услуги по установке программного обеспечения'

In [ ]:
df_res['text_prep'] = prep_text

In [ ]:
df_res.head(10)

,sentiment,okpd_2,name,number,fz,date,text_prep
0,Не подходит,62.01.29.000: Оригиналы программного обеспечен...,Предоставление неисключительного права испол...,№0169300040423000063,44-ФЗ,04.09.2023,предоставление неисключительного права исполь...
1,Не подходит,62.02.20.190: Услуги консультативные в области...,Предоставление услуг по сопровождению програм...,№0162300011123000065,44-ФЗ,07.09.2023,предоставление услуг по сопровождению програм...
2,Не подходит,62.01.29.000: Оригиналы программного обеспечен...,Программное средство антивирусной защиты,№0122100010223000021,44-ФЗ,18.09.2023,программное средство антивирусной защиты
3,Не подходит,62.01.29.000: Оригиналы программного обеспечен...,Программное средство антивирусной защиты,№0302300050523000034,44-ФЗ,21.08.2023,программное средство антивирусной защиты
4,Не подходит,62.01.29.000: Оригиналы программного обеспечен...,Программное средство антивирусной защиты,№0316400001123000094,44-ФЗ,17.08.2023,программное средство антивирусной защиты
5,Не подходит,62.01.29.000: Оригиналы программного обеспечен...,Программное средство антивирусной защиты,№0853300008723000019,44-ФЗ,07.08.2023,программное средство антивирусной защиты
6,Не подходит,62.01.29.000: Оригиналы программного обеспечен...,Программное средство антивирусной защиты,№0853300008723000015,44-ФЗ,04.08.2023,программное средство антивирусной защиты
7,Не подходит,62.01.29.000: Оригиналы программного обеспечен...,Программное средство антивирусной защиты,№0853300008723000014,44-ФЗ,04.08.2023,программное средство антивирусной защиты
8,Не подходит,62.09.20.190: Услуги по технической поддержке ...,Услуги по предоставлению в аренду серверных м...,№0803300221323000001,44-ФЗ,09.09.2023,услуги по предоставлению в аренду серверных м...
9,Не подходит,62.02.30.000: Услуги по технической поддержке ...,\tМодернизация процессов управления автоматизи...,№32312790631,223-ФЗ,24.09.2023,модернизация процессов управления автоматизир...


### Стемминг

In [ ]:
stemmer = SnowballStemmer("russian") 

In [ ]:
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...', 'т.д.', 'т', 'д'])

In [ ]:
text = df_res['text_prep'][0]
word_tokenize(text) 

['предоставление',
 'неисключительного',
 'права',
 'использования',
 'антивирусного',
 'программного',
 'обеспечения',
 'kaspersky',
 'endpoint',
 'security',
 'для',
 'бизнеса',
 'стандартный',
 'russian',
 'edition']

In [ ]:
stemmed_texts_list = []
for text in tqdm(df_res['text_prep']):
    tokens = word_tokenize(text)    
    stemmed_tokens = [stemmer.stem(token) for token in tokens if token not in russian_stopwords]
    text = " ".join(stemmed_tokens)
    stemmed_texts_list.append(text)

df_res['text_stem'] = stemmed_texts_list

C:\Users\Mi\AppData\Local\Temp\ipykernel_9816\863283713.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for text in tqdm(df_res['text_prep']):


  0%|          | 0/3184 [00:00<?, ?it/s]

In [ ]:
def remove_stop_words(text):
    tokens = word_tokenize(text) 
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    return " ".join(tokens)

In [ ]:
sw_texts_list = []
for text in tqdm(df_res['text_prep']):
    tokens = word_tokenize(text)    
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    text = " ".join(tokens)
    sw_texts_list.append(text)

df_res['text_sw'] = sw_texts_list

C:\Users\Mi\AppData\Local\Temp\ipykernel_9816\4291506641.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for text in tqdm(df_res['text_prep']):


  0%|          | 0/3184 [00:00<?, ?it/s]

In [ ]:
df_res['text_sw'][355]

'оказание году услуг адаптации сопровождению экземпляров справочно правовых систем консультант плюс'

In [ ]:
df_res.to_csv('df_1t_stemmed.csv')

In [ ]:
df_res['text_stem'][355]

'оказан год услуг адаптац сопровожден экземпляр справочн правов сист консультант плюс'

## Лемматизация

In [ ]:
# 40 минут выполения, далее сохраним датасет с лемматизацией

lemm_texts_list = []
for text in tqdm(df_res['text_sw']):
    
    try:
        text_lem = mystem.lemmatize(text)
        tokens = [token for token in text_lem if token != ' ' and token not in russian_stopwords]
        text = " ".join(tokens)
        lemm_texts_list.append(text)
    except Exception as e:
        print(e)
    
df_res['text_lemm'] = lemm_texts_list

C:\Users\Mi\AppData\Local\Temp\ipykernel_9816\645692593.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for text in tqdm(df_res['text_sw']):


  0%|          | 0/3184 [00:00<?, ?it/s]

Для ускорения выполнения кода с помощью функции  `lru_cache`  из библиотеки  `functools` , вам нужно создать декоратор и применить его к функции  `mystem.lemmatize()` . 





df_res['text_lemm'] = lemm_texts_list
В этом примере мы создаем декоратор  `lemmatize_text()` , который применяется к функции  `mystem.lemmatize()` . Декоратор  `lru_cache`  кэширует результаты вызовов функции, что позволяет избежать повторных вычислений для одних и тех же входных данных.

Обратите внимание, что декоратор  `lru_cache`  имеет аргумент  `maxsize` , который определяет максимальный размер кэша. Установка  `maxsize=None`  означает, что кэш будет неограниченного размера. Вы можете установить конкретное значение  `maxsize` , чтобы ограничить использование памяти кэша.

Применение декоратора  `lru_cache`  к функции  `mystem.lemmatize()`  позволит кэшировать результаты и избежать повторных вызовов для одних и тех же текстовых данных, что должно ускорить выполнение кода.

In [ ]:
# После выполнения этого кода одни NaN !!!!!!


# from functools import lru_cache

# @lru_cache(maxsize=None)
# def lemmatize_text(text):
#     return mystem.lemmatize(text)

# lemm_texts_list = []
# for text in tqdm(df_res['text_sw']):
#     try:
#         text_lem = lemmatize_text(text)
#         tokens = [token for token in text_lem if token != ' ' and token not in russian_stopwords]
#         text = " ".join(tokens)
#         lemm_texts_list.append(text)
#     except Exception as e:
#         print(e)
        
# # df_res['text_lemm'] = lemm_texts_list     # выдал ошибку ValueError: Length of values (0) does not match length of index (3184)

# # Этот код добавит пустые значения ( `None` ) в конец списка  `lemm_texts_list`  
# # до тех пор, пока его длина не будет соответствовать длине индекса DataFrame  `df_res` .
# df_res['text_lemm'] = lemm_texts_list + [None] * (len(df_res) - len(lemm_texts_list))

In [ ]:
def lemmatize_text(text):
    text_lem = mystem.lemmatize(text)
    tokens = [token for token in text_lem if token != ' ']
    return " ".join(tokens)

In [ ]:
df_res.to_csv('lemm_1t.csv')

In [ ]:
df_res = pd.read_csv('lemm_1t.csv', encoding = 'utf-8')

In [ ]:
df_res.head()

,Unnamed: 0,sentiment,okpd_2,name,number,fz,date,text_prep,text_stem,text_sw,text_lemm
0,0,Не подходит,62.01.29.000: Оригиналы программного обеспечен...,Предоставление неисключительного права испол...,№0169300040423000063,44-ФЗ,04.09.2023,предоставление неисключительного права исполь...,предоставлен неисключительн прав использован а...,предоставление неисключительного права использ...,предоставление неисключительный право использо...
1,1,Не подходит,62.02.20.190: Услуги консультативные в области...,Предоставление услуг по сопровождению програм...,№0162300011123000065,44-ФЗ,07.09.2023,предоставление услуг по сопровождению програм...,предоставлен услуг сопровожден программн компл...,предоставление услуг сопровождению программног...,предоставление услуга сопровождение программны...
2,2,Не подходит,62.01.29.000: Оригиналы программного обеспечен...,Программное средство антивирусной защиты,№0122100010223000021,44-ФЗ,18.09.2023,программное средство антивирусной защиты,программн средств антивирусн защит,программное средство антивирусной защиты,программный средство антивирусный защита \n
3,3,Не подходит,62.01.29.000: Оригиналы программного обеспечен...,Программное средство антивирусной защиты,№0302300050523000034,44-ФЗ,21.08.2023,программное средство антивирусной защиты,программн средств антивирусн защит,программное средство антивирусной защиты,программный средство антивирусный защита \n
4,4,Не подходит,62.01.29.000: Оригиналы программного обеспечен...,Программное средство антивирусной защиты,№0316400001123000094,44-ФЗ,17.08.2023,программное средство антивирусной защиты,программн средств антивирусн защит,программное средство антивирусной защиты,программный средство антивирусный защита \n


In [ ]:
# df_res = df_res.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [ ]:
print(df_res['sentiment'].unique(), len(df_res['sentiment'].unique()))

['Не подходит' 'Подходит' 'Скорее нет' 'Скорее да' 'Затрудняюсь'] 5


In [ ]:
fig = px.histogram(df_res, x='sentiment')
fig.show()

Борьба с дисбалансом классов

При использовании метода upsampling с категориальными значениями, нужно убедиться, что увеличиваемая выборка минорного класса содержит разнообразные значения категориальных переменных, чтобы сохранить разнообразие в данных.

Важно также учитывать, что при увеличении выборки минорного класса с категориальными значениями, может возникнуть риск переобучения модели, особенно если количество увеличенных примеров слишком велико по сравнению с исходным размером минорного класса. Поэтому рекомендуется балансировать выборку с учетом этого фактора.


In [ ]:
X = df_res['text_lemm']
# X = df_res['text_sw']
y = df_res['sentiment']

In [ ]:
fig = px.histogram(df_res, x='sentiment')
fig.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = 42)

In [ ]:
X_train.shape

(2865,)

In [ ]:
y_train.shape

(2865,)

In [ ]:
X_test.shape

(319,)

In [ ]:
y_test.shape

(319,)

## Naive Bayes Classifier

In [ ]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [ ]:
# %%time
nb.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [ ]:
# %%time
y_pred = nb.predict(X_test)

In [ ]:
keyword = 'Подходит'
filtered_df = df_res[df_res['sentiment'].str.contains(keyword, case=False)]

In [ ]:
X_test

1029    оказание услуга предоставление неисключительны...
1001    оказание услуга технический поддержка абонентс...
785     оказание услуга технический сопровождение прог...
411     оказание комплекс услуга защита информация атт...
1105                 оказание услуга защита информация \n
                              ...                        
736     оказание услуга аттестация выделять помещение ...
599     оказание услуга адаптация сопровождение экземп...
2778    оказание услуга обеспечение защита автоматизир...
266     модернизация обеспечение третий линия техничес...
3008    выполнение работа создание сайт профориентация \n
Name: text_lemm, Length: 319, dtype: object

In [ ]:
print(X_test[0], y_test[0], y_pred[0])

предоставление неисключительный право использование антивирусный программный обеспечение kaspersky endpoint security бизнес стандартный russian edition 
 Не подходит Не подходит


In [ ]:
my_tags = df_res['sentiment'].unique()

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred, target_names=my_tags))

accuracy 0.9153605015673981
              precision    recall  f1-score   support

 Не подходит       0.00      0.00      0.00         1
    Подходит       0.95      0.96      0.96       292
  Скорее нет       0.52      0.52      0.52        21
   Скорее да       0.00      0.00      0.00         4
 Затрудняюсь       0.00      0.00      0.00         1

    accuracy                           0.92       319
   macro avg       0.30      0.30      0.30       319
weighted avg       0.91      0.92      0.91       319



c:\Users\Mi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\Mi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\Mi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



## Linear Support Vector Machine

In [ ]:
sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])

In [ ]:
# %%time
sgd.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=42,
                               tol=None))])

In [ ]:
# %%time
y_pred = sgd.predict(X_test)

In [ ]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.9310344827586207
              precision    recall  f1-score   support

 Не подходит       0.00      0.00      0.00         1
    Подходит       0.94      1.00      0.97       292
  Скорее нет       0.75      0.29      0.41        21
   Скорее да       0.00      0.00      0.00         4
 Затрудняюсь       0.00      0.00      0.00         1

    accuracy                           0.93       319
   macro avg       0.34      0.26      0.28       319
weighted avg       0.91      0.93      0.91       319



c:\Users\Mi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\Mi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\Mi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



-------------------------------------------------

### Logistic Regression

In [ ]:
logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5, class_weight='balanced')),
               ])

In [ ]:
# %%time
logreg.fit(X_train, y_train)

c:\Users\Mi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 LogisticRegression(C=100000.0, class_weight='balanced',
                                    n_jobs=1))])

In [ ]:
# %%time
y_pred = logreg.predict(X_test)

In [ ]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.9153605015673981
              precision    recall  f1-score   support

 Не подходит       0.00      0.00      0.00         1
    Подходит       0.95      0.96      0.96       292
  Скорее нет       0.52      0.52      0.52        21
   Скорее да       0.00      0.00      0.00         4
 Затрудняюсь       0.00      0.00      0.00         1

    accuracy                           0.92       319
   macro avg       0.30      0.30      0.30       319
weighted avg       0.91      0.92      0.91       319



c:\Users\Mi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\Mi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\Mi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



### Проверка на примерах

In [ ]:
podhodit = '''
№ 101 Оказание услуг по сопровождению Государственной информационной системы Удмуртской Республики «Управление финансово-экономической деятельностью государственных органов Удмуртской Республики, органов местного самоуправления в Удмуртской Республике и подведомственных им учреждений» для нужд автономного учреждения «Центр цифровых технологий Удмуртской Республики»
'''

In [ ]:
podhodit = remove_multiple_spaces(remove_numbers(remove_punctuation(podhodit.lower())))
podhodit = remove_stop_words(podhodit)

In [ ]:
podhodit

'№ оказание услуг сопровождению государственной информационной системы удмуртской республики управление финансово экономической деятельностью государственных органов удмуртской республики органов местного самоуправления удмуртской республике подведомственных учреждений нужд автономного учреждения центр цифровых технологий удмуртской республики'

In [ ]:
# %%time
podhodit = lemmatize_text(podhodit)

In [ ]:
podhodit

'№ оказание услуга сопровождение государственный информационный система удмуртский республика управление финансовый экономический деятельность государственный орган удмуртский республика орган местный самоуправление удмуртский республика подведомственный учреждение нужда автономный учреждение центр цифровой технология удмуртский республика \n'

In [ ]:
podhodit_pred = logreg.predict([podhodit])
podhodit_pred

array(['Подходит'], dtype=object)

In [ ]:
skoree_da = '''
Закупка комплекса услуг по технической поддержке информационных технологий для нужд Администрации Одинцовского городского округа Московской области в 2023 году
'''

In [ ]:
skoree_da = remove_multiple_spaces(remove_numbers(remove_punctuation(skoree_da.lower())))
skoree_da = remove_stop_words(skoree_da)
skoree_da = lemmatize_text(skoree_da)

In [ ]:
skoree_da_pred = logreg.predict([skoree_da])
skoree_da_pred

array(['Скорее да'], dtype=object)

In [ ]:
skoree_net = '''
Оказание услуг по абонентскому обслуживанию программы ЭВМ на базе информационной бизнес-платформы ODANT «Информационная система управления программой проведения капитального ремонта многоквартирных домов Рязанской области» 
'''

In [ ]:
skoree_net = remove_multiple_spaces(remove_numbers(remove_punctuation(skoree_net.lower())))
skoree_net = remove_stop_words(skoree_net)
skoree_net = lemmatize_text(skoree_net)

In [ ]:
skoree_net_pred = logreg.predict([skoree_net])
skoree_net_pred

array(['Скорее нет'], dtype=object)